## Ricerca bond

In [165]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import numpy_financial as npf

def scarica_rendimenti_eod():
    url = "https://www.simpletoolsforinvestors.eu/documentivari.php"
    
    try:
        response = requests.get(url)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Trova tutti i link con testo 'Link'
        links = soup.find_all('a', string='Link')
        
        # Prendi il settimo link (indice 6)
        if len(links) >= 7:
            file_url = links[6]['href']
            # Costruisci l'URL completo se necessario
            if not file_url.startswith('http'):
                file_url = 'https://www.simpletoolsforinvestors.eu/' + file_url
                
            df = pd.read_csv(file_url, sep=';')
            return df
        else:
            print("Non sono stati trovati abbastanza link nella pagina")
            return None
            
    except Exception as e:
        print(f"Errore durante l'elaborazione: {str(e)}")
        return None
    
    
def calcola_guadagno_netto(df, data_target, giorni_min, giorni_max, importo_investito, issuer=None, costo_acquisto=0):
    # Previous code remains the same until the filtering section
    
    df = df.copy()
    
    # Calculate giorni_investimento and guadagno_netto as before
    
    # Filter by issuer if specified, otherwise keep all
    if issuer is not None:
        df = df[
            (df['issuerdescription'].fillna('ALL') == issuer) |
            (df['issuerdescription'].isna())
        ]
    # Converti le colonne numeriche
    df['netyieldtomaturity'] = pd.to_numeric(df['netyieldtomaturity'].str.replace(',', '.'), errors='coerce')
    df['minimumlot'] = pd.to_numeric(df['minimumlot'], errors='coerce')
    
    # Converti le date
    data_target = pd.to_datetime(data_target)
    df['redemptiondate'] = pd.to_datetime(df['redemptiondate'])
    
    # Calcola l'intervallo di date valide per il filtro
    data_max = data_target - timedelta(days=giorni_min)
    data_min = data_target - timedelta(days=giorni_max)
    
    # Filtra i bond
    mask = (
        (df['redemptiondate'] >= data_min) & 
        (df['redemptiondate'] <= data_max) &
        (df['minimumlot'] <= importo_investito) &
        (df['netyieldtomaturity'].notna()) &
        (df['currencycode'] == 'EUR') &
        (df['minimumlot'] == 1000) &
        (df['volumevalue'] > 1)
    )
    
    bond_filtrati = df.loc[mask].copy()
    
    # Calcola i giorni effettivi di investimento da oggi alla scadenza
    oggi = datetime.now()
    bond_filtrati.loc[:, 'giorni_investimento'] = (bond_filtrati['redemptiondate'] - oggi).dt.days
    
    # Calcola il guadagno netto basato sui giorni di investimento da oggi
    bond_filtrati.loc[:, 'guadagno_netto'] = (
        importo_investito * 
        (bond_filtrati['netyieldtomaturity'] / 100) * 
        (bond_filtrati['giorni_investimento'] / 365) - costo_acquisto
    )
    
    # Arrotonda i risultati
    bond_filtrati.loc[:, 'guadagno_netto'] = bond_filtrati['guadagno_netto'].round(2)
    bond_filtrati.loc[:, 'netyieldtomaturity'] = bond_filtrati['netyieldtomaturity'].round(3)
    
    # Fill NA values in issuerdescription with 'ALL'
    bond_filtrati.loc[:, 'issuerdescription'] = bond_filtrati['issuerdescription'].fillna('ALL')
    
    # Select only the desired columns
    colonne_output = [
        'isincode', 'description', 'redemptiondate', 'issuerdescription', 
        'ratingsp', 'netyieldtomaturity', 'giorni_investimento', 'guadagno_netto'
    ]
    
    bond_filtrati = bond_filtrati[colonne_output]
    
    # Group by issuerdescription and select highest guadagno_netto
    bond_migliori = bond_filtrati.loc[
        bond_filtrati.groupby('issuerdescription')['guadagno_netto'].idxmax()
    ]
    
    return bond_migliori.sort_values('guadagno_netto', ascending=False)


In [166]:
# Esempio di utilizzo
data_target = '2025-06-01'
giorni_min = 5
giorni_max = 30
importo_investito = 1000

risultati = calcola_guadagno_netto(df, data_target, giorni_min, giorni_max, importo_investito, costo_acquisto=5)
risultati

,isincode,description,redemptiondate,issuerdescription,ratingsp,netyieldtomaturity,giorni_investimento,guadagno_netto
370,FR0012517027,"FRANCIA 25/05/2025 0,5%",2025-05-25,Francia,AA,2.337,145,4.28
514,IT0005327306,"BTP 15/05/2025 1,45%",2025-05-15,Italia,BBB,2.202,135,3.14


In [167]:
def seleziona_migliori_bond(df, scadenze_importi, giorni_min, giorni_max, issuer=None, costo_acquisto=0):
    """
    Seleziona il miglior bond per ogni scadenza e importo
    """
    risultati = []
    for data_target, importo in scadenze_importi.items():
        # Seleziona il miglior bond per questa combinazione
        bond_migliore = calcola_guadagno_netto(
            df, data_target, giorni_min, giorni_max, importo, issuer, costo_acquisto
        )
        if not bond_migliore.empty:
            # Prendi solo il bond con il guadagno netto più alto
            miglior_bond = bond_migliore.nlargest(1, 'guadagno_netto')
            # Aggiungi la colonna 'importo_investito'
            miglior_bond['importo_investito'] = importo
            risultati.append(miglior_bond)
    
    # Concatena tutti i risultati
    if risultati:
        return pd.concat(risultati)
    return pd.DataFrame()

def calcola_metriche_investimento(migliori_bond):
    """
    Calcola il rendimento totale, percentuale, durata media in giorni e l'IRR annualizzato
    """
    if migliori_bond.empty:
        return {
            'rendimento_totale': 0,
            'rendimento_percentuale': 0,
            'durata_media_giorni': 0,
            'irr_annualizzato': 0
        }

    # Calcola il rendimento totale
    rendimento_totale = migliori_bond['guadagno_netto'].sum()
    
    # Calcola il rendimento percentuale
    investimento_totale = migliori_bond['importo_investito'].sum()
    rendimento_percentuale = (rendimento_totale / investimento_totale) * 100

    # Calcola la durata media in giorni
    durata_media_giorni = migliori_bond['giorni_investimento'].mean()

    # Calcola IRR
    flussi = [-investimento_totale]  # Flusso iniziale negativo
    for _, bond in migliori_bond.iterrows():
        flusso_entrata = bond['importo_investito'] + bond['guadagno_netto']
        flussi.append(flusso_entrata)

    try:
        irr = npf.irr(flussi)
        # Annualizziamo l'IRR
        irr_annualizzato = ((1 + irr) ** (365 / durata_media_giorni) - 1) * 100
    except:
        irr_annualizzato = 0

    return {
        'rendimento_totale': rendimento_totale,
        'rendimento_percentuale': rendimento_percentuale,
        'durata_media_giorni': durata_media_giorni,
        'irr_annualizzato': irr_annualizzato
    }



In [170]:
scadenze_importi = {
    '2025-03-15': 5000,
    '2025-06-30': 5000,
    '2025-09-30': 5000
}

# Seleziona i migliori bond
migliori_bond= seleziona_migliori_bond(df, scadenze_importi, 5, 40, issuer="Italia", costo_acquisto=5)
metriche = calcola_metriche_investimento(migliori_bond)

for metrica, valore in metriche.items():
    print(f"{metrica}: {valore:.2f}")

migliori_bond

rendimento_totale: 118.75
rendimento_percentuale: 0.79
durata_media_giorni: 155.67
irr_annualizzato: 0.93


,isincode,description,redemptiondate,issuerdescription,ratingsp,netyieldtomaturity,giorni_investimento,guadagno_netto,importo_investito
489,IT0004513641,BTP 01/03/2025 5%(No CACs),2025-03-01,Italia,BBB,2.143,60,12.61,5000
496,IT0005090318,"BTP 01/06/2025 1,5%",2025-06-01,Italia,BBB,2.140,152,39.57,5000
635,IT0005611659,BOT 12/09/2025 ZC,2025-09-12,Italia,BBB,2.049,255,66.57,5000
